In [27]:
base_folder = "R:\\Downloads\\housing_app_fall25-main\\housing_app_fall25-main"
%cd "{base_folder}"

R:\Downloads\housing_app_fall25-main\housing_app_fall25-main


C:\Users\91889\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [29]:
from pathlib import Path
import os
import sqlite3
import pandas as pd
import tarfile
import urllib.request

# --------------------------------------------------
# Base folder (safe for VS Code / Jupyter)
# --------------------------------------------------
base_folder = (
    Path.cwd().parent
    if Path.cwd().name == "notebooks"
    else Path.cwd()
)

DATASETS_DIR = Path("datasets")
DATA_DIR = Path(f"{base_folder}/data")
TARBALL_PATH = DATA_DIR / "titanic.tgz"


# --------------------------------------------------
# Load Titanic data from titanic.tgz
# --------------------------------------------------
def load_titanic_data():
    print("[1] Checking for titanic.tgz…")
    DATASETS_DIR.mkdir(parents=True, exist_ok=True)
    DATA_DIR.mkdir(parents=True, exist_ok=True)

    if not TARBALL_PATH.is_file():
        print("[1a] File not found. Downloading titanic.tgz…")
        url = "https://github.com/ageron/data/raw/main/titanic.tgz"
        urllib.request.urlretrieve(url, TARBALL_PATH)
        print("[1b] Download completed.")

    print("[2] Extracting titanic.tgz…")
    with tarfile.open(TARBALL_PATH) as tar:
        tar.extractall(path=DATASETS_DIR)

    print("[3] Loading titanic.csv into DataFrame…")
    return pd.read_csv(DATASETS_DIR / "titanic" / "titanic.csv")


# --------------------------------------------------
# Build 3NF SQLite Database (Classification)
# --------------------------------------------------
def build_3nf_sqlite(db_path="./data/titanic.db"):
    print("=== BUILDING 3NF SQLITE DATA MODEL (TITANIC CLASSIFICATION) ===")

    print("\n[STEP 1] Loading CSV into DataFrame…")
    df = load_titanic_data()
    print(f"Loaded {len(df)} rows.")

    # --------------------------------------------------
    # Classification target
    # --------------------------------------------------
    print("\n[STEP 1a] Preparing target variable…")
    df = df.dropna(subset=["Survived"])
    df["Survived"] = df["Survived"].astype(int)

    # --------------------------------------------------
    # Surrogate key
    # --------------------------------------------------
    print("\n[STEP 2] Creating surrogate key passenger_id…")
    df = df.reset_index().rename(columns={"index": "passenger_id"})

    # --------------------------------------------------
    # Dimension table: Sex
    # --------------------------------------------------
    print("\n[STEP 3] Building sex dimension table…")
    sex_dim = (
        df[["Sex"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    sex_dim["sex_id"] = sex_dim.index + 1

    print("\n[STEP 4] Merging sex_id into main DataFrame…")
    df = df.merge(sex_dim, on="Sex", how="left")

    # --------------------------------------------------
    # 3NF DataFrames
    # --------------------------------------------------
    print("\n[STEP 5] Creating 3NF DataFrames…")

    df_sex = sex_dim.rename(columns={"Sex": "name"})[
        ["sex_id", "name"]
    ]

    df_passenger = df[
        ["passenger_id", "Pclass", "Age", "Fare", "sex_id"]
    ]

    df_survival = df[
        ["passenger_id", "SibSp", "Parch", "Survived"]
    ]

    print("3NF DataFrames created.")

    # --------------------------------------------------
    # SQLite database
    # --------------------------------------------------
    print("\n[STEP 6] Creating SQLite database and tables…")
    os.makedirs("./data", exist_ok=True)

    if os.path.exists(db_path):
        print("Existing DB found. Removing…")
        os.remove(db_path)

    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    cur.executescript(
        """
        DROP TABLE IF EXISTS passenger_survival;
        DROP TABLE IF EXISTS passenger;
        DROP TABLE IF EXISTS sex;

        CREATE TABLE sex (
            sex_id INTEGER PRIMARY KEY,
            name TEXT NOT NULL UNIQUE
        );

        CREATE TABLE passenger (
            passenger_id INTEGER PRIMARY KEY,
            Pclass INTEGER NOT NULL,
            Age REAL,
            Fare REAL NOT NULL,
            sex_id INTEGER NOT NULL,
            FOREIGN KEY (sex_id)
                REFERENCES sex(sex_id)
        );

        CREATE TABLE passenger_survival (
            passenger_id INTEGER PRIMARY KEY,
            SibSp INTEGER NOT NULL,
            Parch INTEGER NOT NULL,
            Survived INTEGER NOT NULL,
            FOREIGN KEY (passenger_id)
                REFERENCES passenger(passenger_id)
        );
        """
    )
    print("Tables created.")

    # --------------------------------------------------
    # Insert data
    # --------------------------------------------------
    print("\n[STEP 7] Inserting data into SQLite database…")

    cur.executemany(
        "INSERT INTO sex VALUES (?, ?)",
        list(df_sex.itertuples(index=False, name=None)),
    )

    cur.executemany(
        "INSERT INTO passenger VALUES (?, ?, ?, ?, ?)",
        list(df_passenger.itertuples(index=False, name=None)),
    )

    cur.executemany(
        "INSERT INTO passenger_survival VALUES (?, ?, ?, ?)",
        list(df_survival.itertuples(index=False, name=None)),
    )

    conn.commit()
    conn.close()

    print("\n=== DONE! SQLite DB created at:", db_path, "===\n")


# --------------------------------------------------
# RUN
# --------------------------------------------------
build_3nf_sqlite()


=== BUILDING 3NF SQLITE DATA MODEL (TITANIC CLASSIFICATION) ===

[STEP 1] Loading CSV into DataFrame…
[1] Checking for titanic.tgz…
[2] Extracting titanic.tgz…
[3] Loading titanic.csv into DataFrame…
Loaded 891 rows.

[STEP 1a] Preparing target variable…

[STEP 2] Creating surrogate key passenger_id…

[STEP 3] Building sex dimension table…

[STEP 4] Merging sex_id into main DataFrame…

[STEP 5] Creating 3NF DataFrames…
3NF DataFrames created.

[STEP 6] Creating SQLite database and tables…
Existing DB found. Removing…
Tables created.

[STEP 7] Inserting data into SQLite database…

=== DONE! SQLite DB created at: ./data/titanic.db ===

